In [15]:
# %pip install -U pip setuptools wheel
# %pip install -U spacy
# !python -m spacy download en_core_web_sm

In [16]:
import pandas as pd
import spacy


In [17]:
def extract_autism_info(query):
    # Initialize a spaCy doc
    doc = nlp(query)
    
    # Create a list to store the results
    results = []
    
    # Iterate through the tokens in the query
    for token in doc:
        # Check if the token is a known entity
        if token.ent_type_:
            entity = token.ent_type_
            
            # If the entity is related to autism therapies or evaluations, add it to the results
            if "autism therapy" in entity or "autism evaluation" in entity:
                results.append(token.text)
    
    # Extract the age parameter using entity recognition
    age_param = None
    for token in doc:
        if token.ent_type_ == 'age':
            age_param = token.text
            break
    
    # Append the extracted age parameter to the results
    if age_param:
        results.append(f"Age parameter: {age_param}")
    
    return results

In [18]:

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('../goetschm/maic/asd-hackathon/ASD-Hackathon/big_data_energy/our_data.csv', encoding = "utf-8")
input_files = pd.read_csv('../goetschm/maic/asd-hackathon/ASD-Hackathon/big_data_energy/ns_io.csv', encoding = "utf-8")


In [36]:

# Tokenize the text in the first column of the DataFrame
# texts = [nlp(col) for col in df.columns]
texts = input_files['Input'].apply(lambda x:nlp(x))

# Identify parameters in the text
# params = []
# for doc in texts:
#     for ent in doc.ents:
#         # if ent.label_ == 'place':
#         params.append((ent.text, ent.label_))

params = []
for doc in texts:
    for token in doc:
        params.append((token.text, token.pos_))
        # extract subject
        if (token.dep_=='nsubj'):
            print(token.text)
        # extract object
        elif (token.dep_=='dobj'):
            print(token.text)


0    (A, parent, of, a, 3, -, year, -, old, is, wondering, where, they, can, go, to, get, an, evaluation, for, their, child, who, had, a, high, screening, for, autism, ., The, referral, given, by, the,...
1                           (A, parent, of, a, child, recently, diagnosed, with, autism, is, trying, to, figure, out, what, treatment, options, are, available, to, them, and, where, to, go, for, services)
2                                                      (A, provider, is, looking, for, a, list, of, treatment, providers, for, ABA, /, autism, behavior, therapy, who, treat, children, age, 10, years, old)
3    (A, caregiver, has, a, child, diagnosed, with, autism, and, they, are, currently, on, a, waitlist, for, ABA, /, autism, behavior, therapy, ., They, are, looking, for, a, autism, consultant, to, wo...
4                                                         (A, caregiver, is, looking, for, information, and, support, about, their, child, 's, educational, supports, related, to, a

In [21]:
params

[('A', 'DET'),
 ('parent', 'NOUN'),
 ('of', 'ADP'),
 ('a', 'DET'),
 ('3', 'NUM'),
 ('-', 'PUNCT'),
 ('year', 'NOUN'),
 ('-', 'PUNCT'),
 ('old', 'ADJ'),
 ('is', 'AUX'),
 ('wondering', 'VERB'),
 ('where', 'SCONJ'),
 ('they', 'PRON'),
 ('can', 'AUX'),
 ('go', 'VERB'),
 ('to', 'PART'),
 ('get', 'VERB'),
 ('an', 'DET'),
 ('evaluation', 'NOUN'),
 ('for', 'ADP'),
 ('their', 'PRON'),
 ('child', 'NOUN'),
 ('who', 'PRON'),
 ('had', 'VERB'),
 ('a', 'DET'),
 ('high', 'ADJ'),
 ('screening', 'NOUN'),
 ('for', 'ADP'),
 ('autism', 'NOUN'),
 ('.', 'PUNCT'),
 ('The', 'DET'),
 ('referral', 'NOUN'),
 ('given', 'VERB'),
 ('by', 'ADP'),
 ('the', 'DET'),
 ('pediatrician', 'NOUN'),
 ('is', 'AUX'),
 ('a', 'DET'),
 ('long', 'ADJ'),
 ('wait', 'NOUN'),
 ('and', 'CCONJ'),
 ('they', 'PRON'),
 ('want', 'VERB'),
 ('other', 'ADJ'),
 ('options', 'NOUN'),
 ('.', 'PUNCT'),
 ('A', 'DET'),
 ('parent', 'NOUN'),
 ('of', 'ADP'),
 ('a', 'DET'),
 ('child', 'NOUN'),
 ('recently', 'ADV'),
 ('diagnosed', 'VERB'),
 ('with', 'ADP'),